# Crawling PTA Trunojoyo

## Install Library BeautifulSoup

Beautiful Soup adalah sebuah perangkat yang umumnya digunakan untuk melakukan crawling dan ekstraksi informasi dari dokumen HTML dan XML. Alat ini menawarkan metode yang sederhana untuk menjelajahi dan mengolah struktur dokumen HTML. Dengan Beautiful Soup, Anda dapat dengan mudah mencari elemen berdasarkan tag, atribut, dan teks yang terdapat dalam elemen tersebut.

In [ ]:
!pip install beautifulsoup4

## Import Library

In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Crawling Function

Fungsi ini mengambil informasi dari halaman tugas akhir berdasarkan URL yang diberikan. Fungsi tersebut mengekstrak judul tugas akhir, nama penulis, nama dosen pembimbing, dan abstrak dari halaman tersebut.

In [ ]:
def get_data(url):
  data_page = []
  response = requests.get(url)
  if response.status_code == 200:
    page = BeautifulSoup(response.content)
    container_content = page.find('li', {"data-id" : "id-1"})
    header_page = container_content.find('div', {"style" : "float:left; width:540px;"})

    #  Mengambil judul TA
    get_title = header_page.find('a', class_ = 'title').text.strip().split('\r\n')
    title = " ".join(get_title)
    data_page.append(title)

    # Penulis dan dosen pembimbing
    creator = header_page.find_all('span')
    for i in creator:
      splitting_text = i.text.strip().split(':')
      if splitting_text[0].strip() == 'Penulis':
        data_page.append(splitting_text[1].strip())
      elif splitting_text[0].strip() == 'Dosen Pembimbing I':
        data_page.append(splitting_text[1].strip())
      else:
        data_page.append(splitting_text[1].strip())

    # Mengambil Abstrak
    get_abstrak = container_content.find('p', {"align" : "justify"}).text.strip().split('\r\n')
    abstrak = " ".join(get_abstrak)
    data_page.append(abstrak)

  return data_page


Fungsi ini melakukan crawling atau pengambilan data dari halaman yang berisi daftar tugas akhir. Proses ini melibatkan pengambilan URL dari halaman tersebut, pencarian elemen-elemen dalam daftar tugas akhir, ekstraksi URL untuk setiap halaman detail tugas akhir, dan penerapan fungsi get_data() untuk mendapatkan informasi lebih lanjut.

In [ ]:
def crawling_pta(url):
  hasil_crawling = []
  response = requests.get(url)
  if response.status_code == 200:
    page = BeautifulSoup(response.content)
    container_content = page.find('ul', class_= 'items')
    list_content = container_content.find_all('li')

    for content in list_content:
      view_button = content.find('a', class_ = 'gray button').get('href')
      hasil_crawling.append(get_data(view_button))

    next_page = page.find_all('a', class_='pag_button')
    for i in next_page:
      if i.text.strip() == '>':
        return (hasil_crawling, i.get('href'))

Bagian ini mengeksekusi fungsi main dengan menyediakan URL dari halaman awal dan menentukan jumlah halaman yang ingin diambil.

In [ ]:
def main(url, num_page):
    try:
        content, next_page = crawling_pta(url)

        for i in range(num_page - 1):
            more_content, next_page = crawling_pta(next_page)
            content += more_content

        with open('/content/drive/MyDrive/PPW /hasil_crawling/crawling.csv', mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['Judul', 'Penulis', 'Dosen Pembimbing I', 'Dosen Pembimbing II', 'Abstrak'])
            writer.writerows(content)

        print(f'Data Di Simpan Ke crawling_pta.csv Dengan Jumlah : {len(content)} Data')

    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

## Data disimpan ke csv

In [ ]:
main('https://pta.trunojoyo.ac.id/c_search/byprod/10', 10)

Data Di Simpan Ke crawling_pta.csv Dengan Jumlah : 50 Data
